<a href="https://colab.research.google.com/github/PedroAlmeida18/PGP3_TESTESIND/blob/main/Fun%C3%A7%C3%B5esPerdas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Funções de perdas


In [3]:
import torch
from torch import nn
if torch.cuda.is_available():
    device = torch.device('cuda')
else :
  device = torch.device('CPU')


print (device)



cuda


In [4]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target
print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']


In [5]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax()

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output  = self.softmax(self.out(feature))

    return output

input_size  = data.shape[1]
hidden_size = 32
out_size    = len(wine.target_names)

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


#Classificação

In [6]:
criterion = nn.CrossEntropyLoss().to(device)
#transformação de arrays em tensores
Xtns= torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)
#cAST NA GPU
Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(Xtns.dtype,Ytns.dtype)

torch.float32 torch.int64


In [8]:
pred = net(Xtns)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [13]:
print(pred.shape,Ytns.shape)
print(pred[0].data, Ytns[0].data)

torch.Size([178, 3]) torch.Size([178])
tensor([1.0000e+00, 1.6825e-40, 2.7803e-39], device='cuda:0') tensor(0, device='cuda:0')


In [15]:
loss = criterion(pred[:30],Ytns[:30])
print(loss)

tensor(0.5514, device='cuda:0', grad_fn=<NllLossBackward0>)


#Regressão

In [16]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target
print(data.shape,target.shape)
print(data[0])
print(target[0])

(442, 10) (442,)
[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990749 -0.01764613]
151.0


Mlp


In [17]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output  = self.softmax(self.out(feature))

    return output

input_size  = data.shape[1]
hidden_size = 32
out_size    = 1 #progressão da diabetes

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU

WineClassifier(
  (hidden): Linear(in_features=10, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=1, bias=True)
  (softmax): Softmax(dim=None)
)


In [18]:
criterion= nn.MSELoss().to(device)
Xtns= torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)
print(Xtns.shape, Ytns.shape)

torch.Size([442, 10]) torch.Size([442])


In [21]:
#Floward da rede
pred = net(Xtns)

loss = criterion(pred.squeeze(),Ytns)
print(loss.data)


tensor(28771.2168, device='cuda:0')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
